In [1]:

import cv2
import random 
import matplotlib.pyplot as plt
import shutil
from patchify import patchify
import os
import numpy as np
random.seed(42)


In [2]:
N_SAMPLES = 32
PATCH_SIZE = (256,256)
STEP=256

DATASET_PATH= "/Users/amalahmadinurov/Desktop/BP/tissue-cells"
OUTPUT_PATH = "/Users/amalahmadinurov/Desktop/Subjects_ZS_2025/ZNEUS/tiger-patches/"


In [3]:



def convert_to_patches(img_path, mask_path, img_output_path, filename, images_output_folder_path,masks_output_folder_path, counters, image_filenames, mask_filenames):

   
    img = cv2.imread(img_path)
    mask = cv2.imread(mask_path,cv2.IMREAD_GRAYSCALE)
    
    img_patches = patchify(img, (PATCH_SIZE[0], PATCH_SIZE[1],3), step=STEP)
    mask_patches = patchify(mask, (PATCH_SIZE[0], PATCH_SIZE[1]), step=STEP)

    for i in range(0, mask_patches.shape[0]):
        for j in range(0, mask_patches.shape[1]):
            mask_patch = mask_patches[i, j]
            img_patch = img_patches[i,j,0]
            uniq_values, counts = np.unique(mask_patch, return_counts=True)

            patch_filename = f"img_patch_{i}_{j}{filename}"
         
        

           
            
            cv2.imwrite(os.path.join(images_output_folder_path,  patch_filename), img_patch)
            cv2.imwrite(os.path.join(masks_output_folder_path,  patch_filename), mask_patch)
            image_filenames.append(patch_filename)
            mask_filenames.append(patch_filename)




In [5]:





IMAGES_PATH = os.path.join(DATASET_PATH, "images")
MASKS_PATH = os.path.join(DATASET_PATH, "masks")


def create_dataset(image_paths, mask_paths, output_folder_name):
  
    dataset_image_filenames = []
    dataset_mask_filenames = []

    

    images_output_folder_path = os.path.join(OUTPUT_PATH, output_folder_name, "images")
    masks_output_folder_path = os.path.join(OUTPUT_PATH, output_folder_name, "masks")

   

    if not(os.path.isdir(images_output_folder_path)):
        os.makedirs(images_output_folder_path)
    if not(os.path.isdir(masks_output_folder_path)):
        os.makedirs(masks_output_folder_path)
       


    for filename in mask_paths:
    
        img_path = os.path.join(IMAGES_PATH, filename)
        mask_path = os.path.join(MASKS_PATH, filename)
        
        img = cv2.imread(mask_path)
        
        if(img.shape[0] > 1000) or img.shape[1] > 1000:

            convert_to_patches(img_path, mask_path, OUTPUT_PATH, filename, images_output_folder_path, masks_output_folder_path, {}, dataset_image_filenames, dataset_mask_filenames)
            continue

        uniq_values, counts = np.unique(img, return_counts=True)
        percentages = counts / counts.sum() * 100

     
    
    



        shutil.copy(img_path, os.path.join(images_output_folder_path, filename))
        shutil.copy(mask_path, os.path.join(masks_output_folder_path, filename))
        
        dataset_image_filenames.append(filename)
        dataset_mask_filenames.append(filename)

        
    return dataset_image_filenames, dataset_mask_filenames



In [6]:
original_dataset_image_paths = os.listdir(IMAGES_PATH)
original_dataset_mask_paths = os.listdir(MASKS_PATH)

In [7]:
patch_image_filenames, patch_mask_filenames = create_dataset(original_dataset_image_paths, original_dataset_mask_paths, "patches")